In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-28")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
spark.sql (
    f"""
        CREATE TABLE IF NOT EXISTS f1_presentation.calculated_race_results
        (
            race_year INTEGER,
            team_name STRING, 
            driver_id INTEGER,
            driver_name STRING, 
            race_id INTEGER, 
            position INTEGER, 
            points INTEGER,
            calculated_points INTEGER,
            created_date TIMESTAMP,
            updated_date TIMESTAMP
        )
        USING DELTA
    """
)

In [0]:
spark.sql(
        f"""
            CREATE OR REPLACE TEMP VIEW race_result_updated
            AS
            SELECT 
            races.race_year,
            constructors.name AS team_name,
            drivers.driver_id,
            drivers.name AS driver_name,
            results.race_id,
            results.position,
            results.points,
            (11 - results.position) AS calculated_points
            FROM f1_processed.results
            JOIN f1_processed.races ON (results.race_id = races.race_id)
            JOIN f1_processed.drivers ON (results.driver_id = drivers.driver_id)  
            JOIN f1_processed.constructors ON (results.constructor_id = constructors.constructor_id)
            WHERE results.position <= 10 AND results.file_date = '{v_file_date}'
""")

In [0]:
%sql
MERGE INTO f1_presentation.calculated_race_results AS tgt
USING race_result_updated AS src
ON (tgt.driver_id = src.driver_id AND tgt.race_id = src.race_id)
WHEN MATCHED THEN
UPDATE SET
  tgt.position = src.position,
  tgt.points = src.points,
  tgt.calculated_points = src.calculated_points,
  tgt.updated_date = current_timestamp
WHEN NOT MATCHED THEN 
INSERT (race_year, team_name, driver_id, driver_name, race_id, position, points, calculated_points, created_date)
VALUES (race_year, team_name, driver_id, driver_name, race_id, position, points, calculated_points, current_timestamp)       